In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization,Conv1D,Flatten,MaxPooling1D,LSTM
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler


Part 1: Get the data
We used Yahoo Finance to get the data for Tesal stock from 2017/1/6 to 2022/1/5. Our analysis is daily-based, and all the decisions are made using the open price on each day.

For a reason which will be clarified by the following code, our analysis will start from 50 days after January 6, 2017 and end the day before January 5, 2022.

We used Yahoo Finance and DataReader to obtain and load trading data for Tesla stock.

In [2]:
# Identify stock data to grab by ticker
ticker = 'TSLA'

start_date=datetime.datetime(2017,1,6)
end_date=datetime.datetime(2022,1,5)

# df=pd.read_csv("TSLA-2.csv")
# df.index=pd.to_datetime(df["Date"])
# df=df.drop("Date",axis=1)

In [3]:
# For reading stock data from yahoo
from pandas_datareader.data import DataReader

df = DataReader(ticker, 'yahoo', start_date, end_date)

df.drop("Adj Close",axis=1,inplace=True) # May have to adjut columns later

ModuleNotFoundError: No module named 'pandas_datareader'

Now for each day we have the closing price for the day, the closing price of the previous day and the open price of the following day.

The feature rapp is the quotient between the previous day's close and today's closing price. It will be used because it gives the variation (return) of the portfolio for the day.

In [ ]:
df['Prev_Close']=df['Close'].shift(1)
df.head()

In [ ]:
df["rapp"]=df["Close"].divide(df['Close'].shift(1)) # Should be the close of the previous close

In [ ]:
df.head(10)

In [ ]:
print(df.head())
print(df.tail())

In [ ]:
df["mv_avg_short"]= df["Close"].rolling(window=5).mean()
df["mv_avg_long"]= df["Close"].rolling(window=50).mean()

In [ ]:
df.head()

In [ ]:
print(df.loc["2020-12","mv_avg_short"])
print(df.loc["2019-12":"2020-11","Open"])
print(df.loc["2019-12":"2020-11","Open"].mean())

We remove the first 50 days, since they do not have the 50-day moving average

In [ ]:
df=df.iloc[50:,:] # WARNING: DO IT JUST ONE TIME!
print(df.index)

Finally, we can divide df in train and test set

In [ ]:
mtest=300
train=df.iloc[:-mtest,:] 
test=df.iloc[-mtest:,:] 

In [ ]:
print(len(train))
print(len(test))
print(len(df))

Part 2: Define functions to compute gross yield
Notice that the gross yield can be computed very easily using the feature rapp. The following function explains how: the vector v selects which days we are going to stay in the market

In [ ]:
# This function returns the total percentage gross yield and the annual percentage gross yield

def yield_gross(df,v):
    prod=(v*df["rapp"]+1-v).prod()
    n_years=len(v)/252
    return (prod-1)*100,((prod**(1/n_years))-1)*100

Part 3: Define the LSTM model

In [ ]:
def create_window(data, window_size = 1):    
    data_s = data.copy()
    for i in range(window_size):
        data = pd.concat([data, data_s.shift(-(i + 1))], axis = 1)
        
    data.dropna(axis=0, inplace=True)
    return(data)

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
dg=pd.DataFrame(scaler.fit_transform(df[["High","Low","Open","Close","Volume",\
                                          "mv_avg_short","mv_avg_long"]].values))
dg0=dg[[0,1,2,3,4,5]]


window=4
dfw=create_window(dg0,window)

X_dfw=np.reshape(dfw.values,(dfw.shape[0],window+1,6))

y_dfw=np.array(dg[6][window:]) # The Fix

In [ ]:
X_trainw=X_dfw[:-mtest-1,:,:]
X_testw=X_dfw[-mtest-1:,:,:]
y_trainw=y_dfw[:-mtest-1]
y_testw=y_dfw[-mtest-1:]


In [ ]:
def model_lstm(window,features):
    
    model=Sequential()
#     model.add(LSTM(600, input_shape = (window,features), return_sequences=True))
#     model.add(Dropout(0.5))
    model.add(LSTM(300, input_shape = (window,features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(200, input_shape=(window,features), return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(100,kernel_initializer='uniform',activation='relu'))        
    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
    model.compile(loss='mse',optimizer='adam')
    
    
    return model

In [ ]:
model=model_lstm(window+1,6)
history=model.fit(X_trainw,y_trainw,epochs=100, batch_size=30, validation_data=(X_testw, y_testw), \
                  verbose=1, callbacks=[],shuffle=False) # Batch size should be no more than the square root of the # of training rows

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
y_pr=model.predict(X_trainw)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(y_trainw, label="actual")
plt.plot(y_pr, label="prediction")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Actual open price and pedicted one on train set",fontsize=25)
plt.show()

In [ ]:
y_pred=model.predict(X_testw)

In [ ]:
v=np.diff(y_pred.reshape(y_pred.shape[0]),1)
v_lstm=np.maximum(np.sign(v),0)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(y_testw, label="actual")
plt.plot(y_pred, label="prediction")
plt.plot(v_lstm,label="In and out")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Actual open price, predicted one and vector v_lstm",fontsize=25)
plt.show()

Part 4: Compare the LSTM method with other methods
Now we can copare our LSTM-trading-strategy with the buy and hold strategy and the moving average strategy. In order to do so we compute the corresponding vectors v_bh and v_ma which select the days during which we are going to stay in the market.

In [ ]:
v_bh=np.ones(test.shape[0])
v_ma=test["Open"]>test["mv_avg_short"]
v_ma_l=test["Open"]>test["mv_avg_long"]

In [ ]:
def gross_portfolio(df,w):
    portfolio=[ (w*df["rapp"]+(1-w))[:i].prod() for i in range(len(w))]
    return portfolio

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(gross_portfolio(test,v_bh),label="Portfolio Buy and Hold")
plt.plot(gross_portfolio(test,v_ma),label="Portfolio 5-Day Moving Average")
plt.plot(gross_portfolio(test,v_ma_l),label="Portfolio 50-Day Moving Average")
plt.plot(gross_portfolio(test,v_lstm),label="Portfolio LSTM")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Gross portfolios of three methods", fontsize=25)
plt.show()

In [ ]:
print("Test period of {:.2f} years, from {} to {} \n".format(len(v_bh)/365,str(test.loc[test.index[0],"Open"])[:10],\
      str(test.loc[test.index[-1],"Close"])[:10]))

results0=pd.DataFrame({})
results1=pd.DataFrame({})

# results0["Method"]=["Buy and hold","Moving average","LSTM"]
# results1["Method"]=["Buy and hold","Moving average","LSTM"]

results0["Method"]=["Buy and hold","5-Day Moving average","50-Day Moving average","LSTM"]
results1["Method"]=["Buy and hold","5-Day Moving average","50-Day Moving average","LSTM"]

vs=[v_bh,v_ma,v_ma_l,v_lstm]
results0["Total gross yield"]=[str(round(yield_gross(test,vi)[0],2))+" %" for vi in vs]
results1["Annual gross yield"]=[str(round(yield_gross(test,vi)[1],2))+" %" for vi in vs]

print(results0)
print("\n")
print(results1)